In [ ]:
from visualization_fct import *
from sklearn.mixture import BayesianGaussianMixture

# from bokeh.plotting import output_file, show, save
# from bokeh.mpl import to_bokeh

from bokeh.resources import CDN
from bokeh.embed import file_html

import matplotlib.pyplot as plt  # , mpld3
import pyfits


# for now, no ratio in data (no rates A, B or C in this dataset)

only_flux = True
scale_data = True

hdulist = pyfits.open('../iirc_data/all_data_for_ml.fits')
data = hdulist[1].data

X_flux, X, data_thr, data_fr_en = get_iirc_data(
    data, only_flux=only_flux, scale_data=scale_data, thresholded=True)[:4]

# VBGMM with 3 components: 
#np.random.seed(0)

gmm = BayesianGaussianMixture(n_components=5, random_state=0, alpha_init=0.1, n_init=10).fit(X)
preds = gmm.predict(X)
probs = gmm.predict_proba(X)

data_thr['preds'] = pd.Series(preds).astype("category")

color_key = ["red", "yellow", "blue", "grey", "black", "purple", "pink",
             "brown", "green", "orange"] * 2  # Spectral9
#color_key = color_key[:len(set(preds))+1]

means = gmm.means_

print(np.bincount(preds))
print len(np.bincount(preds))
print(gmm.weights_)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit(X).transform(X)

In [ ]:
data_thr['pca1'] = X_pca[:, 0]
data_thr['pca2'] = X_pca[:, 1]
data_thr['pca3'] = X_pca[:, 2]

means =  gmm.means_
covs = gmm.covariances_
print 'set(preds)=', set(preds)
print 'gmm.weights_=', gmm.weights_

#data5 = pd.DataFrame({name: data_thr[name] for name in ['0', '20', '40', '70', 'preds']})
data5 = pd.DataFrame({name: data_thr[name] for name in ['pca1', 'pca2', 'pca3', 'preds']})

covs_ = np.array([covs[j][[0, 20, 40, 70],:][:, [0, 20, 40, 70]] for j in range(covs.shape[0])])
means_ = np.array([means[j][[0, 20, 40, 70]] for j in range(means.shape[0])])
p = scatter_matrix(data5, covs=covs_, means=means_, color_key=color_key, covs_indices=['pca1', 'pca2', 'pca3'],
                   plot_width=200, plot_height=200, spread=True, datashader=False)
show(p)

In [ ]:
plt.plot(pca.components_.T)
plt.show()

In [ ]:
x = 5
y = 0
covs_xy = [covs[j][[x, y]][:, [x, y]] for j in range(len(covs))]
means_xy = [means[j][[x, y]] for j in range(len(covs))]

p = bokeh_plot_cov(data_thr, covs=covs_xy, means=means_xy,
                   x_name='5',
                   y_name='0',
                   pred_name='preds', title=None,  # try preds -> y
                   plot_width=900, plot_height=300)
show(p)